In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'fruit-and-vegetable-image-recognition:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F952827%2F3173719%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240606%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240606T214917Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D376b2349e33d8ad073416bde939b73b432e52402f4569748aa1ddaf2bcea47954f2fc7a42ce1418ff18c89991c2a8d2a5c6fe88031e934dbfe96fbc63d0211f46339df0107869792142c16b861bcc78e49315a58e75d87d4fd0cb73d428f3ba3e29fc79fe1b7aaba51df63aa8233559550d752b17a65127623bb5a0839569bcebb804d559400cb95ba3ff3be4ed2bc8372cd7da7bc91afb0cbb4413d530a7a1bb68ac2672375f7c2b4d3b5d553f97e0bdaff53694be2a4e19f2323e51120d3b32a5d1b08f96e83f6aa05edc1105c3fec12d91e3450f8fff0165c8655048757d2ec609540b7738615237056973f140f29ace1d738780a10feec25085fd9800395'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[=======                                           ] 329932800 bytes downloaded

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense,Conv2D,Flatten,MaxPooling2D,Input,Dropout,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LeakyReLU
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data_train_path ="/kaggle/input/fruit-and-vegetable-image-recognition/train"
data_test_path="/kaggle/input/fruit-and-vegetable-image-recognition/test"
data_val_path = "/kaggle/input/fruit-and-vegetable-image-recognition/validation"


In [ ]:
img_width = 180
img_height = 180

In [ ]:
data_train = tf.keras.utils.image_dataset_from_directory(
data_train_path,
    shuffle= True,
    image_size= (img_width, img_height),
    batch_size = 32,
    validation_split=False
)

In [ ]:
data_cat = data_train.class_names
#data_cat

In [ ]:
data_val = tf.keras.utils.image_dataset_from_directory(
    data_val_path,
    image_size=(img_width,img_height),
    batch_size=32,
    shuffle=False,
    validation_split=False
)

In [ ]:
data_test = tf.keras.utils.image_dataset_from_directory(
data_test_path,
    image_size=(img_width,img_height),
    batch_size=32,
    shuffle=False,
    validation_split=False
)

In [ ]:
plt.figure(figsize=(10,10))
for image, labels in data_train.take(1):
    for i in range(9):
        plt.subplot(3,3,i+1)
        plt.imshow(image[i].numpy().astype('uint8'))
        plt.title(data_cat[labels[i]])
        plt.axis('off')

In [ ]:
data_train

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D


In [ ]:
from tensorflow.keras.layers import Rescaling

# Define the rescaling layer
rescaling = Rescaling(1./255)

# Define the Sequential model
model = Sequential([
    rescaling,  # Rescaling layer
    Conv2D(16, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dropout(0.2),
    Dense(128),
    Dense(len(data_cat)),
])


In [ ]:
    model.compile(optimizer='adam', loss =tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])


In [ ]:
epochs =25
history = model.fit(data_train, validation_data=data_val,epochs=epochs,batch_size=32,verbose=1 )

In [ ]:
epochs_range = range(epochs)

plt.figure(figsize=(8,8))
plt.subplot(2,2,1)
plt.plot(epochs_range, history.history['accuracy'],label='Training Accuracy')
plt.plot(epochs_range , history.history['val_accuracy'],label ='Validation Accuracy')
plt.title('Accuracy')

plt.subplot(2,2,2)
plt.plot(epochs_range , history.history['loss'], label='Training Loss')
plt.plot(epochs_range, history.history['val_loss'], label='Validation Loss')
plt.title('Loss')

plt.show()

In [ ]:
#image 'Enter your fruit image to classify'
#image= tf.keras.utils.load_img(image, target_size=(img_width, img_height))
#img_arr = tf.keras.utils.array_to_img(image)
#img_bat = tf.expand_dims(img_arr, 0)

In [ ]:
#predict = model.predict(img_bat)

In [ ]:
#score= tf.nn.softmax(predict)

In [ ]:
#print('Veg / Fruit in Image is {}  with accuracy of {0.2f}'.format(data_cat[np.argmax(score),np.max(score)*100]) )